# 📈 مشروع ESRGAN لتحسين دقة الصور باستخدام بيانات DIV2K (محلية)
دفتر Jupyter احترافي لتحميل البيانات، بناء النموذج، تدريبه، وتقييم الأداء باستخدام PSNR و SSIM.

## 1. استيراد المكتبات

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image
from PIL import Image
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

## 2. تحميل بيانات DIV2K من المجلد المحلي

In [ ]:

# ⚠️ قم بتعديل المسارات حسب جهازك
lr_dir = '../data/DIV2K_valid_LR_bicubic/X4'
hr_dir = '../data/DIV2K_valid_HR'

# تحويلات مختلفة للـ LR و HR
transform_lr = transforms.Compose([
    transforms.Resize((64, 64)),  # لتتناسب مع ESRGAN input
    transforms.ToTensor()
])

transform_hr = transforms.Compose([
    transforms.Resize((256, 256)),  # لتتناسب مع مخرجات ESRGAN
    transforms.ToTensor()
])

class DIV2KDataset(Dataset):
    def __init__(self, lr_dir, hr_dir, transform_lr=None, transform_hr=None):
        self.lr_dir = lr_dir
        self.hr_dir = hr_dir
        self.transform_lr = transform_lr
        self.transform_hr = transform_hr
        self.files = sorted(os.listdir(lr_dir))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        lr_path = os.path.join(self.lr_dir, self.files[idx])
        hr_path = os.path.join(self.hr_dir, self.files[idx].replace('x4', ''))
        lr_img = Image.open(lr_path).convert('RGB')
        hr_img = Image.open(hr_path).convert('RGB')
        if self.transform_lr:
            lr_img = self.transform_lr(lr_img)
        if self.transform_hr:
            hr_img = self.transform_hr(hr_img)
        return lr_img, hr_img

# تحميل البيانات
dataset = DIV2KDataset(lr_dir, hr_dir, transform_lr=transform_lr, transform_hr=transform_hr)
loader = DataLoader(dataset, batch_size=2, shuffle=True)


## 3. بناء نموذج ESRGAN

In [4]:
class ResidualDenseBlock(nn.Module):
    def __init__(self, nf=64, gc=32):
        super().__init__()
        self.conv1 = nn.Conv2d(nf, gc, 3, 1, 1)
        self.conv2 = nn.Conv2d(nf + gc, gc, 3, 1, 1)
        self.conv3 = nn.Conv2d(nf + 2 * gc, gc, 3, 1, 1)
        self.conv4 = nn.Conv2d(nf + 3 * gc, gc, 3, 1, 1)
        self.conv5 = nn.Conv2d(nf + 4 * gc, nf, 3, 1, 1)
        self.lrelu = nn.LeakyReLU(0.2, inplace=True)

    def forward(self, x):
        x1 = self.lrelu(self.conv1(x))
        x2 = self.lrelu(self.conv2(torch.cat([x, x1], 1)))
        x3 = self.lrelu(self.conv3(torch.cat([x, x1, x2], 1)))
        x4 = self.lrelu(self.conv4(torch.cat([x, x1, x2, x3], 1)))
        x5 = self.conv5(torch.cat([x, x1, x2, x3, x4], 1))
        return x + x5 * 0.2

class RRDB(nn.Module):
    def __init__(self, nf, gc=32):
        super().__init__()
        self.rdb1 = ResidualDenseBlock(nf, gc)
        self.rdb2 = ResidualDenseBlock(nf, gc)
        self.rdb3 = ResidualDenseBlock(nf, gc)

    def forward(self, x):
        return x + self.rdb3(self.rdb2(self.rdb1(x))) * 0.2

class ESRGANGenerator(nn.Module):
    def __init__(self, in_nc=3, out_nc=3, nf=64, nb=5, gc=32):
        super().__init__()
        self.conv_first = nn.Conv2d(in_nc, nf, 3, 1, 1)
        self.rrdb_blocks = nn.Sequential(*[RRDB(nf, gc) for _ in range(nb)])
        self.trunk_conv = nn.Conv2d(nf, nf, 3, 1, 1)
        self.upconv1 = nn.Conv2d(nf, nf, 3, 1, 1)
        self.upconv2 = nn.Conv2d(nf, nf, 3, 1, 1)
        self.conv_last = nn.Conv2d(nf, out_nc, 3, 1, 1)
        self.lrelu = nn.LeakyReLU(0.2, inplace=True)

    def forward(self, x):
        fea = self.conv_first(x)
        trunk = self.trunk_conv(self.rrdb_blocks(fea))
        fea = fea + trunk
        fea = self.lrelu(F.interpolate(self.upconv1(fea), scale_factor=2))
        fea = self.lrelu(F.interpolate(self.upconv2(fea), scale_factor=2))
        return self.conv_last(fea)

## 4. تدريب النموذج

In [8]:
from tqdm import tqdm  # لعرض شريط تقدم التدريب

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ESRGANGenerator().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.L1Loss()

epochs = 3  # مثلاً 3 تكرارات
for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    for lr_imgs, hr_imgs in tqdm(loader, desc=f"Epoch {epoch+1}/{epochs}"):
        lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
        preds = model(lr_imgs)
        loss = loss_fn(preds, hr_imgs)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"✅ نهاية Epoch {epoch+1}, متوسط الخسارة: {epoch_loss / len(loader):.4f}")


Epoch 1/3:   0%|          | 0/50 [00:08<?, ?it/s]


RuntimeError: The size of tensor a (1024) must match the size of tensor b (256) at non-singleton dimension 3

## 5. تقييم النموذج وعرض النتائج

In [ ]:
model.eval()
lr_img, hr_img = dataset[0]
with torch.no_grad():
    input_tensor = lr_img.unsqueeze(0).to(device)
    output = model(input_tensor).squeeze().cpu().clamp(0, 1)

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.imshow(to_pil_image(lr_img))
plt.title('Low Resolution')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(to_pil_image(output))
plt.title('Super Resolved')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(to_pil_image(hr_img))
plt.title('High Resolution')
plt.axis('off')

plt.tight_layout()
plt.show()

sr_np = output.permute(1, 2, 0).numpy()
hr_np = hr_img.permute(1, 2, 0).numpy()
print(f"PSNR: {psnr(hr_np, sr_np):.2f}")
print(f"SSIM: {ssim(hr_np, sr_np, multichannel=True):.4f}")